In [1]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [2]:
import sys
sys.path.append("../")

In [3]:
from context_retriever import ContextRetriever


retriever = ContextRetriever()
retriever.set_params(topK_searchEngine = 150, topK_SPLADE = 30, topK_crossEncoder = 5, topK_LLM = None, score_threshold = None, 
                     pick_rag_index = None, use_classifier = False)

# question = "A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?"
# formated_choices = ""
# context = retriever.get_RAG_context(question, formated_choices)
# print(context)

/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-13 10:21:03,745 - INFO - Use pytorch device_name: cuda:0
2025-10-13 10:21:03,746 - INFO - Load pretrained SparseEncoder: naver/splade-v3


Template validated successfully!
Successfully created pointwise inference handler!


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:

from datasets import load_dataset

dataset = load_dataset("TsinghuaC3I/UltraMedical")["train"]

In [ ]:
medqa_subset = dataset.filter(lambda d: d["id"].startswith("MedQA,"))
print(medqa_subset)

In [5]:
medqa_evol_subset = dataset.filter(lambda d: d["id"].startswith("MedQA-Evol,"))
print(medqa_evol_subset)

Dataset({
    features: ['id', 'type', 'conversations', 'answer', 'score'],
    num_rows: 51809
})


In [6]:

def format_choices(choices):
    final_answers = []
    for i, (x, y) in enumerate(choices.items()):
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)

    
def convert_to_chat_format(conversations, is_input):
    chat = []
    for conversation in conversations:
        if conversation["from"] == "human":
            value = conversation["value"]
            for option_letter in ["A", "B", "C", "D"]:
                value = value.replace(f"{option_letter}.", f"[{option_letter}] :")
            chat.append({"role": "user", "content": value})
        elif is_input == False and conversation["from"] == "gpt":
            value = conversation["value"]
            for option_letter in ["A", "B", "C", "D"]:
                value = value.replace(f"{option_letter}.", f"[{option_letter}].")
            chat.append({"role": "assistant", "content": value})
    return chat


In [7]:
import logging

# Log
# log_file_name = "10-2-MedQA_context_training.txt"
log_file_name = "10-8-MedQA_Evol_context_training.txt"
# log_file_name = "a.txt"

# Remove all handlers associated with the root logger
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set log file config
logging.basicConfig(
    filename = log_file_name,      # Log file name
    filemode = 'a',                    # Append mode
    format = '%(asctime)s - %(levelname)s - %(message)s',
    level = logging.INFO               # Log level
)

In [ ]:
# subset = medqa_subset
# subset = medqa_evol_subset

start_index = 42369
subset = medqa_evol_subset.select(range(start_index, len(medqa_evol_subset)))

for i in range(len(subset)):
    print(f"{i+1}/{len(subset)}")
    logging.info(f"[question index:]{i + start_index}")
    
    data = subset[i]
    questions_and_answers = convert_to_chat_format(data["conversations"], is_input = False)

    question_and_options = questions_and_answers[0]["content"]
    
    logging.info(f"[question_and_options:]{question_and_options}")
    
    # print(questions_and_answers[0]["content"])

    # These few lines of code is for only one document
    doc_data_list = retriever.get_RAG_data_list(question_and_options, "")
    for doc_idx in range(len(doc_data_list)):
        doc_data = doc_data_list[doc_idx]
        doc = doc_data["content"]
        logging.info(f"[rag_doc:]{doc_idx}:{doc}")
        # print(doc)

    # # These few lines of code is for 2 or more documents
    # context = retriever.get_RAG_context(question_and_options, "")
    # logging.info(f"[rag_doc:]{0}:{context}")


    expected_output = questions_and_answers[1]["content"]
    logging.info(f"[expected_output:]{expected_output}")

    # if i > 4:
    #     break

    # question = data["question"]
    # choices = data["options"]
    # print(question, choices)
    # break

1/9440


Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors


2/9440
3/9440
4/9440
5/9440
